In [ ]:
from dotenv import load_dotenv
from IPython.display import display
import os
import pandas as pd
from sklearn.cluster import KMeans
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

pd.set_option('display.max_rows', 1000)

load_dotenv()
client_id=os.getenv('CLIENT_ID')
client_secret=os.getenv('CLIENT_SECRET')

credentials = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=credentials)

def get_playlist_tracks(playlist_id):
    tracks = []
    results = sp.playlist_tracks(playlist_id)
    tracks.extend(results['items'])
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks

def get_track_features(track_ids):
    features = []
    chunk_size = 100
    for i in range(0, len(track_ids), chunk_size):
        chunk = track_ids[i:i + chunk_size]
        features.extend(sp.audio_features(chunk))
    df = pd.DataFrame(features)
    return df

playlist_id = '10eNrlNCbpuVZFiwVcuWOg'
tracks = get_playlist_tracks(playlist_id)

track_ids = [track['track']['id'] for track in tracks if track['track']['id'] is not None]
track_names = [track['track']['name'] for track in tracks if track['track']['id'] is not None]

features = get_track_features(track_ids)
features['name'] = [track['track']['name'] for track in tracks if track['track']['id'] is not None and track['track']['id'] in features['id'].tolist()]

features = features.drop_duplicates(subset=['name'], keep='last', ignore_index=True)
selected_features = features[['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']]

kmeans = KMeans(n_clusters=3, random_state=0)
features['cluster'] = kmeans.fit_predict(selected_features)

for cluster in features['cluster'].unique():
    print(f"Playlist {cluster}:")
    display(features[features['cluster'] == cluster][['name', 'cluster']])